In [1]:
import pandas as pd
import openpyxl
import os
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pyparsing import col
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm 
import re

In [2]:
#검색할 식당 데이터

items = ['로흐', '권선백청우칼국수', '합정피자마루', '순도리']

In [13]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.maximize_window()


count = 0 #
current = 0 #현재 진행 상황

goal = len(items) #총 식당 수

#데이터 프레임으로 만들 빈 리스트 생성
rev_list=[]


for item in items: 
    
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    
    #네이버 지도 검색
    search_url = f'https://map.naver.com/v5/search/{item}/place' 
    driver.get(search_url)
    driver.switch_to.frame("searchIframe")
    time.sleep(3)
    

     #검색한 식당이 있을 때 리뷰 정보 가져오기
    if len(driver.find_elements(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li')) != 0:
        
        print('식당 존재')
        
        #식당 정보 클릭
        driver.execute_script('return document.querySelector("#_pcmap_list_scroll_container > ul > li:nth-child(1) > div.ouxiq.icT4K > a:nth-child(1) > div").click()')
        time.sleep(2)
        
        
        #리뷰 버튼 클릭
        driver.execute_script('return ')
        tabs = driver.window_handles
        time.sleep(2)
        
        # reviewbutton = driver.find_element(by='xpath', value='//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[4]')
        # time.sleep(2)
        # content_url = reviewbutton.get_attribute("href")
        
        #리뷰창이 새 탭에서 뜨도록 처리 
        # tabs = driver.window_handles
        # driver.switch_to.window(tabs[1]) # 새 탭으로 이동
        # driver.get(content_url) # 링크 열기
        # time.sleep(2)
        

        #스크롤 내리기
        for c in range(0,4):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(1)
        
        
        #리뷰 데이터 스크래핑을 위한 html 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.eCPGL > li')
        
        #리뷰 수가 0이 아닌 경우 리뷰글 가져오기
        if len(review_lists) != 0 :
            
            #후기 더보기가 있으면 모두 눌러주기 
            while True: 
                try:
                    if driver.execute_script('return document.querySelector("#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div.lfH3O > a").textContent') == "더보기":
                        driver.execute_script('return document.querySelector("#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div.lfH3O > a > svg").click()')
                        time.sleep(2) 
                        
                except:
                    print("더보기 종료")
                    break 
                
            #후기 더보기를 모두 누른 후 리뷰 리스트 재정의
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            review_lists = soup.select('.eCPGL > li')
            print(len(review_lists))
                    
            
            #리뷰 정보가 있는 경우 리뷰글, 식당 평균 평점, 리뷰 작성 시간 가져오기     
            try:
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        
                        try:
                            #리뷰 내용이 긴 경우에 더보기 눌러주기
                            if driver.find_element(By.CSS_SELECTOR, '.div.ZZ4OK > a > span.rvCSr > span')=="내용 더보기":
                                driver.find_element(By.CSS_SELECTOR, '.div.ZZ4OK > a > span.rvCSr').send_keys(Keys.ENTER)
                            
                            #더보기 누른 후에 리뷰 내용 스크래핑    
                            user_review = review.select(' div.ZZ4OK > a > span') # 리뷰 
                            rate = driver.find_element(by='xpath',value = '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[3]/span[1]/em').text #식당평균평점
                            timestamp = review.select(' div.sb8UA > span:nth-child(1) > span:nth-child(3)') #시간정보
                             
                        
                            #리뷰 정보가 있는 경우 식당 이름, 평점, 리뷰 텍스트, 작성 시간을 가져와서 데이터 프레임으로 만들기
                            rev_list.append(
                            [
                                item,
                                rate,
                                user_review[0].text,
                                timestamp[0].text
                            ]
                            )            
    
                            time.sleep(1)  
                        
                            
                        except:
                            user_review = review.select(' div.ZZ4OK > a > span') # 리뷰
                            rate = driver.find_element(by='xpath',value = '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[3]/span[1]/em').text #식당평균평점
                            timestamp = review.select(' div.sb8UA > span:nth-child(1) > span:nth-child(3)') #시간정보
                        
                            #리뷰 정보가 있는 경우 식당 이름, 평점, 리뷰 텍스트, 작성 시간을 가져와서 데이터 프레임으로 만들기
                            rev_list.append(
                            [
                                item,
                                rate,
                                user_review[0].text,
                                timestamp[0].text
                            ]
                            )            
    
                            time.sleep(1)  
                            
                
               
            except:
                print("더 이상 리뷰가 존재하지 않습니다.")
                
                
                
        else :
            #리뷰 정보가 없는 경우 식당명과 평균 평점만 데이터 프레임에 추가
            rate = driver.find_element(by='xpath',value = '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[3]/span[1]/em').text
            rev_list.append(
                [
                    item,
                    rate,
                ]
            ) 

                    
            print("리뷰가 없습니다")
        
        #검색한 창 닫고 카카오맵 검색 페이지로 돌아가기    
        driver.close()
        driver.switch_to.window(tabs[0])
        print("기본 페이지로 돌아가기")
            
    else:
        print("식당이 존재하지 않음")
        
driver.close()

#스크래핑한 데이터를 데이터 프레임으로 만들기    
columns = ["name", "rate", "review", "timestamp"]
df = pd.DataFrame(rev_list, columns=columns)
df    

진행상황 : 1/4
식당 존재


InvalidArgumentException: Message: invalid argument: invalid locator
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x003B1ED3+2236115]
	Ordinal0 [0x003492F1+1807089]
	Ordinal0 [0x002566FD+812797]
	Ordinal0 [0x002850F5+1003765]
	Ordinal0 [0x002857CB+1005515]
	Ordinal0 [0x002B7632+1209906]
	Ordinal0 [0x002A1AD4+1120980]
	Ordinal0 [0x002B59E2+1202658]
	Ordinal0 [0x002A18A6+1120422]
	Ordinal0 [0x0027A73D+960317]
	Ordinal0 [0x0027B71F+964383]
	GetHandleVerifier [0x0065E7E2+2743074]
	GetHandleVerifier [0x006508D4+2685972]
	GetHandleVerifier [0x00442BAA+532202]
	GetHandleVerifier [0x00441990+527568]
	Ordinal0 [0x0035080C+1837068]
	Ordinal0 [0x00354CD8+1854680]
	Ordinal0 [0x00354DC5+1854917]
	Ordinal0 [0x0035ED64+1895780]
	BaseThreadInitThunk [0x74DC7BA9+25]
	RtlInitializeExceptionChain [0x76F2BB3B+107]
	RtlClearBits [0x76F2BABF+191]
